Demanda por zona y mes – Pickup

In [ ]:
with base as (
  select
      t.year,
      t.month,
      dz.zone_id,
      dz.borough,
      dz.zone,
      count(*) as trips
  from GOLD.FCT_TRIPS t
  left join GOLD.DIM_ZONE dz on dz.zone_sk = t.pu_zone_sk
  where t.pickup_datetime between '2019-01-01' and '2019-12-31'
  group by 1,2,3,4,5
),
ranked as (
  select *,
         row_number() over (partition by year, month order by trips desc) as rn
  from base
)
select year, month, zone_id, borough, zone, trips
from ranked
where rn <= 10
order by year, month, trips desc;


Demanda por zona y mes – Dropoff

In [ ]:
with base as (
  select
      t.year,
      t.month,
      dz.zone_id,
      dz.borough,
      dz.zone,
      count(*) as trips
  from GOLD.FCT_TRIPS t
  left join GOLD.DIM_ZONE dz on dz.zone_sk = t.do_zone_sk
  where t.pickup_datetime between '2019-01-01' and '2019-12-31'
  group by 1,2,3,4,5
),
ranked as (
  select *,
         row_number() over (partition by year, month order by trips desc) as rn
  from base
)
select year, month, zone_id, borough, zone, trips
from ranked
where rn <= 10
order by year, month, trips desc;


Ingresos por borough y mes (PU)

In [ ]:
select
    dz.borough,
    t.year,
    t.month,
    sum(t.total_amount) as revenue_usd
from GOLD.FCT_TRIPS t
left join GOLD.DIM_ZONE dz on dz.zone_sk = t.pu_zone_sk
where t.pickup_datetime between '2019-01-01' and '2019-12-31'
group by 1,2,3
order by 2,3,1;


In [ ]:
select
  dz.borough,
  t.year,
  t.month,
  sum(t.total_amount)                                  as revenue_gross_usd,
  sum(coalesce(t.tip_amount,0))                        as tips_usd,
  sum(t.total_amount) - sum(coalesce(t.tip_amount,0))  as revenue_net_ex_tip_usd,
  round(nullif(sum(coalesce(t.tip_amount,0)),0)
        / nullif(sum(t.total_amount),0) * 100, 2)      as tip_pct
from GOLD.FCT_TRIPS t
left join GOLD.DIM_ZONE dz on dz.zone_sk = t.pu_zone_sk
where t.pickup_datetime >= '2019-01-01'
  and t.pickup_datetime <  '2020-01-01'
group by 1,2,3
order by 2,3,1;


Velocidad promedio (mph) por franja horaria

In [ ]:
with base as (
  select
      dz.borough,
      case
          when extract(hour from t.pickup_datetime) between 6 and 21 then 'day'
          else 'night'
      end as band,
      sum(t.trip_distance) as sum_miles,
      sum(t.trip_minutes)  as sum_minutes
  from GOLD.FCT_TRIPS t
  left join GOLD.DIM_ZONE dz on dz.zone_sk = t.pu_zone_sk
  where t.trip_distance is not null
    and t.trip_minutes  > 0
    and t.pickup_datetime between '2019-01-01' and '2019-12-31'
  group by 1,2
)
select
  borough,
  band,
  round(sum_miles / nullif(sum_minutes,0) * 60, 2) as avg_mph
from base
order by borough, band;


Duración del viaje – percentiles

In [ ]:
select
    dz.zone_id as pu_location_id,
    approx_percentile(t.trip_minutes, 0.5) as p50_minutes,
    approx_percentile(t.trip_minutes, 0.9) as p90_minutes,
    count(*) as trips
from GOLD.FCT_TRIPS t
left join GOLD.DIM_ZONE dz on dz.zone_sk = t.pu_zone_sk
where t.trip_minutes > 0
  and t.trip_minutes < 240
  and t.pickup_datetime between '2019-01-01' and '2019-12-31'
group by dz.zone_id
order by trips desc;

Elasticidad temporal – distribución por hora/día

In [ ]:
select
    dayofweek(t.pickup_datetime) as dow,     -- 1=Domingo
    extract(hour from t.pickup_datetime) as hh,
    count(*) as trips
from GOLD.FCT_TRIPS t
where t.pickup_datetime between '2019-01-01' and '2019-12-31'
group by 1,2
order by 1,2;


Horas pico

In [ ]:
with by_hour as (
  select
      extract(hour from t.pickup_datetime) as hh,
      count(*) as trips
  from GOLD.FCT_TRIPS t
  where t.pickup_datetime between '2019-01-01' and '2019-12-31'
  group by 1
)
select *
from by_hour
order by trips desc
limit 10;